In [1]:
from __future__ import print_function
import os
import argparse
from glob import glob

from PIL import Image
import tensorflow as tf

from model import lowlight_enhance
from utils import *

parser = argparse.ArgumentParser(description='')

parser.add_argument('--use_gpu', dest='use_gpu', type=int, default=1, help='gpu flag, 1 for GPU and 0 for CPU')
parser.add_argument('--gpu_idx', dest='gpu_idx', default="0", help='GPU idx')
parser.add_argument('--gpu_mem', dest='gpu_mem', type=float, default=0.5, help="0 to 1, gpu memory usage")
parser.add_argument('--phase', dest='phase', default='test', help='train or test')

parser.add_argument('--epoch', dest='epoch', type=int, default=100, help='number of total epoches')
parser.add_argument('--batch_size', dest='batch_size', type=int, default=16, help='number of samples in one batch')
parser.add_argument('--patch_size', dest='patch_size', type=int, default=48, help='patch size')
parser.add_argument('--start_lr', dest='start_lr', type=float, default=0.001, help='initial learning rate for adam')
parser.add_argument('--eval_every_epoch', dest='eval_every_epoch', default=20, help='evaluating and saving checkpoints every #  epoch')
parser.add_argument('--checkpoint_dir', dest='ckpt_dir', default='./NewCheckpoint/', help='directory for checkpoints')
parser.add_argument('--sample_dir', dest='sample_dir', default='./BrightNetSample/', help='directory for evaluating outputs')

parser.add_argument('--save_dir', dest='save_dir', default='./BicubicInterpolation/', help='directory for testing outputs')
parser.add_argument('--test_dir', dest='test_dir', default='./Result_test_folder/low/', help='directory for testing inputs')
parser.add_argument('--decom', dest='decom', default=0, help='decom flag, 0 for enhanced results only and 1 for decomposition results')

args, unknown = parser.parse_known_args()
print(args.use_gpu)
def lowlight_train(lowlight_enhance):
    if not os.path.exists(args.ckpt_dir):
        os.makedirs(args.ckpt_dir)
    if not os.path.exists(args.sample_dir):
        os.makedirs(args.sample_dir)

    lr = args.start_lr * np.ones([args.epoch])
    lr[20:] = lr[0] / 10.0

    train_low_data = []
    train_high_data = []

    train_low_data_names = glob('./New_LOLDataset/our485/low/*.png') 
    train_low_data_names.sort()
    train_high_data_names = glob('./New_LOLDataset/our485/high/*.png')
    train_high_data_names.sort()
    assert len(train_low_data_names) == len(train_high_data_names)
    print('[*] Number of training data: %d' % len(train_low_data_names))
    
    for idx in range(len(train_low_data_names)):
        low_im = load_images(train_low_data_names[idx])
        train_low_data.append(low_im)
        high_im = load_images(train_high_data_names[idx])
        train_high_data.append(high_im)

    eval_low_data = []
    eval_high_data = []

    eval_low_data_name = glob('./New_LOLDataset/eval15/low/*.png')
    for idx in range(len(eval_low_data_name)):
        eval_low_im = load_images(eval_low_data_name[idx])
        eval_low_data.append(eval_low_im)

    lowlight_enhance.train(train_low_data, train_high_data, eval_low_data, batch_size=args.batch_size, patch_size=args.patch_size, epoch=args.epoch, lr=lr, sample_dir=args.sample_dir, ckpt_dir=os.path.join(args.ckpt_dir, 'Decom'), eval_every_epoch=args.eval_every_epoch, train_phase="Decom")

    lowlight_enhance.train(train_low_data, train_high_data, eval_low_data, batch_size=args.batch_size, patch_size=args.patch_size, epoch=args.epoch, lr=lr, sample_dir=args.sample_dir, ckpt_dir=os.path.join(args.ckpt_dir, 'Relight'), eval_every_epoch=args.eval_every_epoch, train_phase="Relight")


def lowlight_test(lowlight_enhance):
    if args.test_dir == None:
        print("[!] please provide --test_dir")
        exit(0)

    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    test_low_data_name = glob(os.path.join(args.test_dir) + '/*.*')
    test_low_data = []
    test_high_data = []
    for idx in range(len(test_low_data_name)):
        test_low_im = load_images(test_low_data_name[idx])
        test_low_data.append(test_low_im)

    lowlight_enhance.test(test_low_data, test_high_data, test_low_data_name, save_dir=args.save_dir, decom_flag=args.decom)


def main(_):
    if args.use_gpu:
        print("[*] GPU\n")
        os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_idx
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=args.gpu_mem)
        with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
            model = lowlight_enhance(sess)
            if args.phase == 'train':
                lowlight_train(model)
            elif args.phase == 'test':
                print("in"),lowlight_test(model)
            else:
                print('[!] Unknown phase')
                exit(0)
    else:
        print("[*] CPU\n")
        with tf.Session() as sess:
            model = lowlight_enhance(sess)
            if args.phase == 'train':
                lowlight_train(model)
            elif args.phase == 'test':
                lowlight_test(model)
            else:
                print('[!] Unknown phase')
                exit(0)

if __name__ == '__main__':
    tf.app.run()


1
[*] GPU



W0508 15:48:38.226703 127319431334128 module_wrapper.py:139] From /mnt/Extended_RetinexNet_rakesh/RetinexNet/model.py:84: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0508 15:48:38.235446 127319431334128 module_wrapper.py:139] From /mnt/Extended_RetinexNet_rakesh/RetinexNet/model.py:19: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0508 15:48:38.236071 127319431334128 module_wrapper.py:139] From /mnt/Extended_RetinexNet_rakesh/RetinexNet/model.py:19: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0508 15:48:38.237327 127319431334128 deprecation.py:323] From /mnt/Extended_RetinexNet_rakesh/RetinexNet/model.py:20: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0508 15:48:38.241809 127319431334128 deprecation.py:323] From /opt/anaconda/envs/w

[*] Initialize model successfully...
in


I0508 15:48:41.608906 127319431334128 saver.py:1284] Restoring parameters from ./NewCheckpoint/Decom/RetinexNet-Decom-9100
I0508 15:48:41.707237 127319431334128 saver.py:1284] Restoring parameters from ./NewCheckpoint/Relight/RetinexNet-Relight-9100


[*] Reading checkpoint...
[*] Load weights successfully...
[*] Testing...
./Result_test_folder/low/1.png
./Result_test_folder/low/111.png
./Result_test_folder/low/146.png
./Result_test_folder/low/179.png
./Result_test_folder/low/2015_02686.JPG
./Result_test_folder/low/2015_02859.jpg
./Result_test_folder/low/2015_02899.jpg
./Result_test_folder/low/22.png
./Result_test_folder/low/23.png
./Result_test_folder/low/493.png
./Result_test_folder/low/547.png
./Result_test_folder/low/55.png
./Result_test_folder/low/665.png
./Result_test_folder/low/669.png
./Result_test_folder/low/748.png
./Result_test_folder/low/778.png
./Result_test_folder/low/780.png
./Result_test_folder/low/79.png
./Result_test_folder/low/r000da54ft.png
./Result_test_folder/low/r0a808003t.png
./Result_test_folder/low/r0b6e4254t.png
./Result_test_folder/low/r102b4aa4t.png
./Result_test_folder/low/r113ee9edt.png
./Result_test_folder/low/r133bb357t.png


SystemExit: 

/opt/anaconda/envs/wmlce/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
